### Build the CSV of pedestal values (HV & NoHV) for FPGA 209 (PB 008) and FPGA 210 (PB 06)

In [23]:
import json
from pathlib import Path
import pandas as pd

In [24]:
base = Path("/home/lorena/Documents/PhD/EEEMCal_Analysis/Pedestal_Stability/Data/Pedestals")
files = {
    # ("209", "008", "NoHV"): base / "103_PedestalCalib_pedecalib_20251017_132119.json",
    # ("209", "008", "HV"):   base / "103_PedestalCalib_pedecalib_20251017_174324.json",
    # ("210", "06",  "NoHV"): base / "103_PedestalCalib_pedecalib_20251017_132121.json",
    # ("210", "06",  "HV"):   base / "103_PedestalCalib_pedecalib_20251017_174322.json",
    ("208", "06",  "HV"): "/home/lorena/Documents/PhD/EEEMCal_Analysis/Linearity_Studies_Nov2025/pedestals/103_PedestalCalib_pedecalib_20251103_114212.json",
    ("209", "008", "HV"): "/home/lorena/Documents/PhD/EEEMCal_Analysis/Linearity_Studies_Nov2025/pedestals/103_PedestalCalib_pedecalib_20251103_114212.json",
}

In [25]:
# Channels to exclude completely (1-based indexing)
exclude_channels = {19, 38, 57, 76, 95, 114, 133}

# Helper: load pede_values into a dict {channel_number: pedestal}
def load_pedestals(json_path):
    with open(json_path, "r") as f:
        data = json.load(f)
    pede_values = data.get("pede_values", [])
    # Build 1..151 mapping (skip channel 0, and ignore >151 if present)
    mapping = {ch: pede_values[ch] for ch in range(1, min(len(pede_values), 152)) if ch not in exclude_channels}
    return mapping

In [26]:
# Build long dataframe: one row per (FPGA, PB, Channel, Condition, Pedestal)
rows = []
for (fpga, pb, cond), path in files.items():
    ped = load_pedestals(path)
    for ch, val in ped.items():
        rows.append({
            "FPGA": fpga,
            "PB": pb,
            "Channel": ch,
            "Condition": cond,
            "Pedestal": val
        })

df_long = pd.DataFrame(rows).sort_values(["FPGA", "PB", "Channel", "Condition"]).reset_index(drop=True)

In [ ]:
# Pivot to get HV and NoHV columns side-by-side per FPGA/PB/Channel
df_wide = df_long.pivot_table(index=["FPGA", "PB", "Channel"], columns="Condition", values="Pedestal").reset_index()
# Ensure column order
if "HV" not in df_wide.columns: df_wide["HV"] = pd.NA

#######################################
## In case I want to compare HV/NoHV ##
#######################################
# if "NoHV" not in df_wide.columns: df_wide["NoHV"] = pd.NA
# df_wide = df_wide[["FPGA", "PB", "Channel", "NoHV", "HV"]].sort_values(["FPGA", "PB", "Channel"]).reset_index(drop=True)

#######################################################
## In case I want to get only the HV pedestal values ##
#######################################################
df_wide = df_wide[["FPGA", "PB", "Channel", "HV"]].sort_values(["FPGA", "PB", "Channel"]).reset_index(drop=True)


In [28]:
# Save CSV
# out_csv = base / "pedestals_HV_vs_NoHV_FPGA209_210.csv"
out_csv = "/home/lorena/Documents/PhD/EEEMCal_Analysis/Linearity_Studies_Nov2025/pedestals/pedestals_HV_modified_FPGA208_209.csv"
df_wide.to_csv(out_csv, index=False)

In [29]:
# Show first 20 rows in Jupyter
display(df_wide.head(20))

# Print output file path
print(f"✅ CSV saved to: {out_csv}")


Condition,FPGA,PB,Channel,HV
0,208,06,1,93.0
1,208,06,2,83.0
2,208,06,3,81.0
3,208,06,4,88.0
4,208,06,5,82.0
5,208,06,6,99.0
6,208,06,7,88.0
7,208,06,8,89.0
8,208,06,9,120.0
9,208,06,10,76.0


✅ CSV saved to: /home/lorena/Documents/PhD/EEEMCal_Analysis/Linearity_Studies_Nov2025/pedestals/pedestals_HV_modified_FPGA208_209.csv


In [30]:
# Verify that each JSON has exactly one pedestal per channel (excluding skipped ones)

for (fpga, pb, cond), path in files.items():
    ped = load_pedestals(path)
    print(f"{ped}")
    unique_channels = len(ped)
    unique_values = len(set(ped.keys()))
    print(f"{fpga=} {pb=} {cond=}: {unique_channels} unique channels mapped")

    # sanity check
    if unique_channels != unique_values:
        print("⚠️  Channel index duplication detected!")
    else:
        print("✅  One-to-one mapping confirmed.")

    # quick check of possible duplicates or length mismatch
    if len(ped) != (151 - len(exclude_channels)):
        print(f"⚠️  Expected {151 - len(exclude_channels)} channels, found {len(ped)}")
    else:
        print("✅  Channel count matches expectation.")
    print("-"*50)

{1: 93.0, 2: 83.0, 3: 81.0, 4: 88.0, 5: 82.0, 6: 99.0, 7: 88.0, 8: 89.0, 9: 120.0, 10: 76.0, 11: 84.0, 12: 85.0, 13: 80.0, 14: 103.0, 15: 80.0, 16: 87.0, 17: 101.0, 18: 82.0, 20: 81.0, 21: 86.0, 22: 83.0, 23: 79.0, 24: 81.0, 25: 85.0, 26: 82.0, 27: 82.0, 28: 81.0, 29: 86.0, 30: 73.0, 31: 79.0, 32: 83.0, 33: 57.0, 34: 86.0, 35: 83.0, 36: 85.0, 37: 0.0, 39: 78.0, 40: 63.0, 41: 81.0, 42: 81.0, 43: 80.0, 44: 80.0, 45: 86.0, 46: 58.0, 47: 85.0, 48: 81.0, 49: 99.0, 50: 77.0, 51: 79.0, 52: 76.0, 53: 67.0, 54: 85.0, 55: 81.0, 56: 80.0, 58: 82.0, 59: 82.0, 60: 85.0, 61: 82.0, 62: 85.0, 63: 84.0, 64: 79.0, 65: 79.0, 66: 90.0, 67: 117.0, 68: 81.0, 69: 77.0, 70: 78.0, 71: 82.0, 72: 82.0, 73: 83.0, 74: 70.0, 75: 0.0, 77: 79.0, 78: 57.0, 79: 84.0, 80: 85.0, 81: 110.0, 82: 77.0, 83: 76.0, 84: 83.0, 85: 78.0, 86: 77.0, 87: 81.0, 88: 81.0, 89: 77.0, 90: 46.0, 91: 81.0, 92: 75.0, 93: 77.0, 94: 80.0, 96: 81.0, 97: 95.0, 98: 78.0, 99: 79.0, 100: 81.0, 101: 85.0, 102: 89.0, 103: 81.0, 104: 78.0, 105: 87.0,